## Setup

### Import Libraries

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

print("Pandas version:", pd.__version__)

Pandas version: 2.3.0


### Import Data

#### Mckinsey GDP Dataset

In [2]:
mk_df = pd.read_csv("data/mckinsey.csv")
mk_df.head(3)

,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710


#### Employees Dataset

In [3]:
employee_data = [
    ["Alex", 31, "Senior SD", "01-06-2021"],
    ["Ajax", 31, "Senior SD", "01-01-2025"],
    ["Anna", 40, "V.P.", "01-08-2000"],
    ["Jane", 28, "Junior SD", "01-03-2023"],
    ["John", 35, "Architect", "01-12-2022"],
    ["Anna", 40, "V.P.", "01-08-2000"],
]

column_names = ["Name", "Age", "Role", "D.O.J"]

emp_df = pd.DataFrame(data=employee_data, columns=column_names)
emp_df.head()

,Name,Age,Role,D.O.J
0,Alex,31,Senior SD,01-06-2021
1,Ajax,31,Senior SD,01-01-2025
2,Anna,40,V.P.,01-08-2000
3,Jane,28,Junior SD,01-03-2023
4,John,35,Architect,01-12-2022


In [4]:
emp_df["D.O.J"] = pd.to_datetime(emp_df["D.O.J"], format="%d-%m-%Y")
emp_df["D.O.J"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 6 entries, 0 to 5
Series name: D.O.J
Non-Null Count  Dtype         
--------------  -----         
6 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 180.0 bytes


## Duplicate Data

### Fetch Duplicates

In [5]:
fltr = emp_df.duplicated()
fltr

0    False
1    False
2    False
3    False
4    False
5     True
dtype: bool

In [6]:
type(fltr)

pandas.core.series.Series

In [7]:
emp_df[fltr]

,Name,Age,Role,D.O.J
5,Anna,40,V.P.,2000-08-01


### Delete Duplicates

#### Entire DataFrame

In [8]:
emp_df.drop_duplicates()

,Name,Age,Role,D.O.J
0,Alex,31,Senior SD,2021-06-01
1,Ajax,31,Senior SD,2025-01-01
2,Anna,40,V.P.,2000-08-01
3,Jane,28,Junior SD,2023-03-01
4,John,35,Architect,2022-12-01


In [9]:
emp_df.drop_duplicates(inplace=True, keep="last")

#### Subset

In [10]:
emp_df["Role"].value_counts()

Role
Senior SD    2
Junior SD    1
Architect    1
V.P.         1
Name: count, dtype: int64

In [11]:
emp_df.drop_duplicates(subset=["Role"])

,Name,Age,Role,D.O.J
0,Alex,31,Senior SD,2021-06-01
3,Jane,28,Junior SD,2023-03-01
4,John,35,Architect,2022-12-01
5,Anna,40,V.P.,2000-08-01


In [12]:
emp_df.drop_duplicates(subset=["Role"], keep="last")

,Name,Age,Role,D.O.J
1,Ajax,31,Senior SD,2025-01-01
3,Jane,28,Junior SD,2023-03-01
4,John,35,Architect,2022-12-01
5,Anna,40,V.P.,2000-08-01


## Concatenation

### Row Concatenation

#### Insert single row

In [13]:
# New employee record.
new_emp = [
    {
        "Name": "Bob",
        "Age": 30,
        "Role": "Junior SD",
        "D.O.J": "01-07-2024",
    }
]

# Create new DataFrame for new employee record.
new_emp_df1 = pd.DataFrame(data=new_emp)
new_emp_df1

,Name,Age,Role,D.O.J
0,Bob,30,Junior SD,01-07-2024


Observe that new DataFrame has explicit index `0`

In [14]:
# Concatenate new DataFrame to main DataFrame.
emp_df = pd.concat([emp_df, new_emp_df1])
emp_df

,Name,Age,Role,D.O.J
0,Alex,31,Senior SD,2021-06-01 00:00:00
1,Ajax,31,Senior SD,2025-01-01 00:00:00
3,Jane,28,Junior SD,2023-03-01 00:00:00
4,John,35,Architect,2022-12-01 00:00:00
5,Anna,40,V.P.,2000-08-01 00:00:00
0,Bob,30,Junior SD,01-07-2024


> **Note**:
>
> If `ignore_index` parameter of `concat` method is not set to `True` then explicit indices from both the DataFrames are retained as it is.

#### Insert multiple rows

In [15]:
new_emps = [
    ["David", 30, "Junior SD", "01-07-2024"],
    ["Karen", 33, "Associate Architect", "01-07-2024"],
]
column_names = ["Name", "Age", "Role", "D.O.J"]

new_emp_df2 = pd.DataFrame(data=new_emps, columns=column_names)
new_emp_df2

,Name,Age,Role,D.O.J
0,David,30,Junior SD,01-07-2024
1,Karen,33,Associate Architect,01-07-2024


In [16]:
# Concatenate new DataFrame to main DataFrame.
emp_df = pd.concat([emp_df, new_emp_df2], axis=0, ignore_index=True)
emp_df

,Name,Age,Role,D.O.J
0,Alex,31,Senior SD,2021-06-01 00:00:00
1,Ajax,31,Senior SD,2025-01-01 00:00:00
2,Jane,28,Junior SD,2023-03-01 00:00:00
3,John,35,Architect,2022-12-01 00:00:00
4,Anna,40,V.P.,2000-08-01 00:00:00
5,Bob,30,Junior SD,01-07-2024
6,David,30,Junior SD,01-07-2024
7,Karen,33,Associate Architect,01-07-2024


Check explicit indices

In [17]:
emp_df.index.values

array([0, 1, 2, 3, 4, 5, 6, 7])

Check implicit indices

In [18]:
np.where(emp_df.index.values)
[emp_df.index.get_loc(exp_idx) for exp_idx in emp_df.index.values]

[0, 1, 2, 3, 4, 5, 6, 7]

> **Note**:
>
> `concat()` method does not have `inplace` parameter.

#### Typecasting

##### Before Typecasting

In [19]:
emp_df["D.O.J"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8 entries, 0 to 7
Series name: D.O.J
Non-Null Count  Dtype 
--------------  ----- 
8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes


In [20]:
emp_df["D.O.J"] = pd.to_datetime(emp_df["D.O.J"], format="%d-%m-%Y")

##### After Typecasting

In [21]:
emp_df["D.O.J"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8 entries, 0 to 7
Series name: D.O.J
Non-Null Count  Dtype         
--------------  -----         
8 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 196.0 bytes


### Column Concatenation

In [22]:
emp_gender = pd.DataFrame(
    data=["M", "M", "F", "M", "F", "M", "M", "F"],
    columns=["Gender"],
)
emp_gender.head(3)

,Gender
0,M
1,M
2,F


In [23]:
pd.concat([emp_df, emp_gender], axis=1)

,Name,Age,Role,D.O.J,Gender
0,Alex,31,Senior SD,2021-06-01,M
1,Ajax,31,Senior SD,2025-01-01,M
2,Jane,28,Junior SD,2023-03-01,F
3,John,35,Architect,2022-12-01,M
4,Anna,40,V.P.,2000-08-01,F
5,Bob,30,Junior SD,2024-07-01,M
6,David,30,Junior SD,2024-07-01,M
7,Karen,33,Associate Architect,2024-07-01,F


In [24]:
# Concatenate new DataFrame to main DataFrame.
emp_df = pd.concat([emp_df, emp_gender], axis=1)
emp_df

,Name,Age,Role,D.O.J,Gender
0,Alex,31,Senior SD,2021-06-01,M
1,Ajax,31,Senior SD,2025-01-01,M
2,Jane,28,Junior SD,2023-03-01,F
3,John,35,Architect,2022-12-01,M
4,Anna,40,V.P.,2000-08-01,F
5,Bob,30,Junior SD,2024-07-01,M
6,David,30,Junior SD,2024-07-01,M
7,Karen,33,Associate Architect,2024-07-01,F


## Merging

Merging is done using Joins

## Aggregation

### Aggregation functions

In [25]:
mk_df.head()

,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710
3,Afghanistan,1967,11537966,Asia,34.020,836.197138
4,Afghanistan,1972,13079460,Asia,36.088,739.981106


In [26]:
mk_df["life_exp"].mean()

np.float64(59.474439366197174)

In [27]:
mk_df["life_exp"].sum()

np.float64(101344.44467999999)

In [28]:
mk_df["life_exp"].count()

np.int64(1704)

In [29]:
mk_df["life_exp"].describe()

count    1704.000000
mean       59.474439
std        12.917107
min        23.599000
25%        48.198000
50%        60.712500
75%        70.845500
max        82.603000
Name: life_exp, dtype: float64

### Sorting

#### Sorting by single column

In [30]:
emp_df.sort_values(by="Age")

,Name,Age,Role,D.O.J,Gender
2,Jane,28,Junior SD,2023-03-01,F
5,Bob,30,Junior SD,2024-07-01,M
6,David,30,Junior SD,2024-07-01,M
0,Alex,31,Senior SD,2021-06-01,M
1,Ajax,31,Senior SD,2025-01-01,M
7,Karen,33,Associate Architect,2024-07-01,F
3,John,35,Architect,2022-12-01,M
4,Anna,40,V.P.,2000-08-01,F


In [31]:
emp_df.sort_values(by="D.O.J", ascending=False)

,Name,Age,Role,D.O.J,Gender
1,Ajax,31,Senior SD,2025-01-01,M
5,Bob,30,Junior SD,2024-07-01,M
6,David,30,Junior SD,2024-07-01,M
7,Karen,33,Associate Architect,2024-07-01,F
2,Jane,28,Junior SD,2023-03-01,F
3,John,35,Architect,2022-12-01,M
0,Alex,31,Senior SD,2021-06-01,M
4,Anna,40,V.P.,2000-08-01,F


#### Sorting by multiple columns

In [32]:
emp_df.sort_values(
    by=["Gender", "Age", "D.O.J"],
    ascending=[True, True, False],
)

,Name,Age,Role,D.O.J,Gender
2,Jane,28,Junior SD,2023-03-01,F
7,Karen,33,Associate Architect,2024-07-01,F
4,Anna,40,V.P.,2000-08-01,F
5,Bob,30,Junior SD,2024-07-01,M
6,David,30,Junior SD,2024-07-01,M
1,Ajax,31,Senior SD,2025-01-01,M
0,Alex,31,Senior SD,2021-06-01,M
3,John,35,Architect,2022-12-01,M


> **Note**:
> 
> Setting the parameter `inplace` to `True` in `sort` function will permanently sort the rows w.r.t selected Series in DataFrame.